In [1]:
# prapare packages
import numpy as np
import tensorflow as tf
import matplotlib as plt

In [2]:
# Normal distribution sampling class
# p_d(x)
class DataDistribution(object):
    # in: mu, sigma
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    # returns N samples of sorted normal N ~ (mu, sigma)
    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma)
        samples.sort()
        return samples

In [ ]:
# p_z(z)
class 